In [ ]:
%env OPENAI_API_KEY=<PUT_YOUR_API_KEY_HERE>


In [9]:
from openai import OpenAI
import os
from langchain.prompts import PromptTemplate
from langchain.llms import openai
from langchain.chains import LLMChain

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

llm = openai.OpenAI(model="text-davinci-003", max_tokens=2048, temperature=0.5)

en_to_fr_prompt = PromptTemplate(
    template="Please translate the following sentence into French: \n\n {question}?", input_variables=["question"]
)

question_prompt = PromptTemplate(
    template = "{english_question}", input_variables=["english_question"]
)

fr_to_en_prompt = PromptTemplate(
    input_variables=["english_answer"],
    template="Please translate the following paragraph into English： \n\n{english_answer}?",
)

question_translate_chain = LLMChain(llm=llm, prompt=en_to_fr_prompt, output_key="english_question")
english = question_translate_chain.run(question="As an expert in machine learning, please introduce the principle of CNN.")
print(english)

qa_chain = LLMChain(llm=llm, prompt=question_prompt, output_key="english_answer")
english_answer = qa_chain.run(english_question=english)
print(english_answer)

answer_translate_chain = LLMChain(llm=llm, prompt=fr_to_en_prompt)
answer = answer_translate_chain.run(english_answer=english_answer)
print(answer)




En tant qu'expert en apprentissage automatique, veuillez présenter le principe de CNN ?


Le réseau neuronal convolutif (CNN) est une technique d'apprentissage automatique qui combine l'apprentissage profond et le traitement des images. Les CNN sont composés de couches de neurones connectés entre eux, qui sont utilisés pour extraire des caractéristiques des images. Les couches d'entrée sont connectées à des couches de convolution, qui appliquent des filtres à l'image pour extraire des caractéristiques. Ces caractéristiques sont ensuite passées à des couches de max-pooling, qui réduisent la taille des images tout en conservant les caractéristiques importantes. Finalement, les données sont envoyées à des couches entièrement connectées, qui sont utilisées pour classer l'image.


Convolutional Neural Network (CNN) is a machine learning technique that combines deep learning and image processing. CNNs are composed of layers of neurons connected to each other, which are used to extract feat

In [10]:
from langchain.chains import SimpleSequentialChain

french_qa_chain = SimpleSequentialChain(
    chains=[question_translate_chain, qa_chain, answer_translate_chain], input_key="question",
    verbose=True)
answer = french_qa_chain.run(question="En tant qu'expert en apprentissage automatique, veuillez présenter le principe du CNN.")
print(answer)




> Entering new SimpleSequentialChain chain...


En tant qu'expert en apprentissage automatique, veuillez présenter le principe du Réseau de Neurones Convolutifs (CNN) ?


Le Réseau de Neurones Convolutifs (CNN) est une technique d'apprentissage automatique qui est largement utilisée pour la reconnaissance d'images et le traitement du langage naturel. Il est basé sur le concept de couches de neurones artificiels qui sont connectées entre elles et qui sont entraînées pour effectuer des tâches spécifiques.

Les CNNs sont composés de plusieurs couches de neurones qui sont connectées entre elles. Chaque couche est responsable de la transformation des données d'entrée en une représentation plus abstraite. Les couches de convolution appliquent des filtres à l'image d'entrée qui extraient des caractéristiques spécifiques. Les couches de pooling combinent ces caractéristiques et les réduisent à une taille plus petite. Les couches entièrement connectées sont ensuite utilisées pour effectuer la

In [14]:
from langchain.chains import SequentialChain

q1_prompt = PromptTemplate(
    input_variables=["year1"],
    template="Which team won the Super Bowl in {year1}? Only the team name is required."
)
q2_prompt = PromptTemplate(
    input_variables=["year2"],
    template="Which team won the Super Bowl in {year2}? Only the team name is required."
)
q3_prompt = PromptTemplate(
    input_variables=["team1", "team2"],
    template="Which team, {team1} or {team2}, has won more Super Bowl championships?"
)
chain1 = LLMChain(llm=llm, prompt=q1_prompt, output_key="team1")
chain2 = LLMChain(llm=llm, prompt=q2_prompt, output_key="team2")
chain3 = LLMChain(llm=llm, prompt=q3_prompt)

sequential_chain = SequentialChain(chains=[chain1, chain2, chain3], input_variables=["year1", "year2"], verbose=True)
answer = sequential_chain.run(year1=2016, year2=2000)
print(answer)




> Entering new SequentialChain chain...

> Finished chain.


The Denver Broncos have won 3 Super Bowl championships, while the St. Louis Rams have won 1.


In [8]:
from langchain.chains import SequentialChain

def write_unit_test(function_to_test, unit_test_package = "unittest"):

    explain_code = """"# How to write great unit tests with {unit_test_package}

    In this advanced tutorial for experts, we'll use Python 3.10 and `{unit_test_package}` to write a suite of unit tests to verify the behavior of the following function.
    ```python
    {function_to_test}
    ```

    Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
    - First,"""

    explain_code_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test"],
        template=explain_code
    )
    explain_code_llm = openai.OpenAI(model="text-davinci-002", temperature=0.4, max_tokens=1000,
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])
    explain_code_step = LLMChain(llm=explain_code_llm, prompt=explain_code_template, output_key="code_explaination")

    test_plan = """

    A good unit test suite should aim to:
    - Test the function's behavior for a wide range of possible inputs
    - Test edge cases that the author may not have foreseen
    - Take advantage of the features of `{unit_test_package}` to make the tests easy to write and maintain
    - Be easy to read and understand, with clean code and descriptive names
    - Be deterministic, so that the tests always pass or fail in the same way

    `{unit_test_package}` has many convenient features that make it easy to write and maintain unit tests. We'll use them to write unit tests for the function above.

    For this particular function, we'll want our unit tests to handle the following diverse scenarios (and under each scenario, we include a few examples as sub-bullets):
    -"""
    test_plan_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination"],
        template= explain_code + "{code_explaination}" + test_plan
    )
    test_plan_llm = openai.OpenAI(model="text-davinci-002", temperature=0.4, max_tokens=1000,
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])
    test_plan_step = LLMChain(llm=test_plan_llm, prompt=test_plan_template, output_key="test_plan")

    prompt_to_generate_the_unit_test = """

Before going into the individual tests, let's first look at the complete suite of unit tests as a cohesive whole. We've added helpful comments to explain what each line does.
```python
import {unit_test_package}  # used for our unit tests

{function_to_test}

"""

    unit_test_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination", "test_plan"],
        template= explain_code + "{code_explaination}" + test_plan + "{test_plan}" + prompt_to_generate_the_unit_test
    )
    unit_test_llm = openai.OpenAI(model="text-davinci-002", temperature=0.4, max_tokens=1000, stop="```")
    unit_test_step = LLMChain(llm=unit_test_llm, prompt=unit_test_template, output_key="unit_test")

    sequential_chain = SequentialChain(chains=[explain_code_step, test_plan_step, unit_test_step],
                                    input_variables=["unit_test_package", "function_to_test"], verbose=True)
    answer = sequential_chain.run(unit_test_package=unit_test_package, function_to_test=function_to_test)
    return answer

code = """
def earth_to_mars_time(earth_seconds):
    # Length of a Martian sol in Earth seconds
    mars_sol_length = 88775.244

    # Convert Earth seconds to Mars seconds
    mars_seconds = earth_seconds * mars_sol_length / 86400

    # Calculate Mars time components
    sols = int(mars_seconds // mars_sol_length)
    mars_seconds %= mars_sol_length
    hours = int(mars_seconds // 3600)
    mars_seconds %= 3600
    minutes = int(mars_seconds // 60)
    seconds = int(mars_seconds % 60)

    return sols, hours, minutes, seconds
"""

import ast

def write_unit_test_automatically(code, retry=3):
    unit_test_code = write_unit_test(code)
    all_code = code + unit_test_code
    tried = 0
    while tried < retry:
        try:
            ast.parse(all_code)
            return all_code
        except SyntaxError as e:
            print(f"Syntax error in generated code: {e}")
            all_code = code + write_unit_test(code)
            tried += 1

print(write_unit_test_automatically(code))


/home/fw/miniconda3/envs/ai_aesthetics/lib/python3.10/site-packages/langchain/utils/utils.py:159: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(
/home/fw/miniconda3/envs/ai_aesthetics/lib/python3.10/site-packages/langchain/utils/utils.py:159: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(
/home/fw/miniconda3/envs/ai_aesthetics/lib/python3.10/site-packages/langchain/utils/utils.py:159: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(




> Entering new SequentialChain chain...

> Finished chain.

def earth_to_mars_time(earth_seconds):
    # Length of a Martian sol in Earth seconds
    mars_sol_length = 88775.244

    # Convert Earth seconds to Mars seconds
    mars_seconds = earth_seconds * mars_sol_length / 86400

    # Calculate Mars time components
    sols = int(mars_seconds // mars_sol_length)
    mars_seconds %= mars_sol_length
    hours = int(mars_seconds // 3600)
    mars_seconds %= 3600
    minutes = int(mars_seconds // 60)
    seconds = int(mars_seconds % 60)

    return sols, hours, minutes, seconds
class TestEarthToMarsTime(unittest.TestCase):
    """
    This class contains our unit tests for the earth_to_mars_time function.
    """

    def test_normal_cases(self):
        """
        This test case tests the function's behavior for a variety of normal inputs.
        """

        # Test 1: The function is called with a positive integer
        self.assertEqual(
            earth_to_mars_time(1000000000